<h1><strong> AUSM 학습용 데이터셋 전처리 </strong></h1>
<br>

<hr>

<br>
<h2> 1. Introduction </h2>
 <p style="font-size:16px">개요 : 본 노트북은 LQ(저화질) 이미지를 HQ(고화질) 이미지로 보정하기 위해 AUSM(Adaptive Unsharp Mask) 학습용 데이터셋을 정제하는 전처리 과정을 기록한 리포트이다.
 <p style="font-size:16px">목표 : 일반적인 HQ 이미지를 다운샘플링 및 블러 처리 하여 LQ 이미지를 생성하고, LQ-HQ 간 패치 별 그리드서치(Grid-Search)를 활용하여 k-map을 추출한다.</p>

<br>
<hr>
<br>
<h2> 2. Library Import </h2>


In [39]:
import os, cv2, glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Tuple, List, Dict

import sys
sys.path.append("../src") 
import params

<hr>
<br>
<h2> 3. 이미지 변환 및 k-map 전처리 함수 </h2>

In [40]:
def list_images(path: str, exts: Tuple[str, ...]) -> List[str]:
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(path, f"*{e}")))
    return sorted(files)

<p style="font-size:16px">
   위 함수는 학습에 사용될 이미지들의 경로와 허용할 확장자들을 입력받아 files 리스트에 저장한다.
</p><br>

In [41]:
def get_sigma(img: np.ndarray) -> float:
    if img is None:
        return 1.0
        
    height, width = img.shape[:2]
    long_side = max(height, width)
    calculated_sigma = long_side / 250.0
    final_sigma = np.clip(calculated_sigma, 1.2, 4.8)
    print("[INFO] 사용된 시그마 : ", float(final_sigma))
    return float(final_sigma)

<p style="font-size:16px">
위 함수는 입력 이미지의 해상도에 맞춰 언샤프 마스킹에 사용할 가우시안 블러의 표준편차 값을 동적으로 계산한다.
이미지가 클수록 블러 반경을 넓게 해야 유의미한 엣지(고주파 부분)를 추출할 수 있기 때문에 추가했다.

1.2 ~ 4.8로만 제한하는 로직도 안전을 위해 추가했다.
</p>

In [42]:
def ensure_3ch(img: np.ndarray) -> np.ndarray:
    if img is None:
        return None

    if img.ndim == 2:
        return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    if img.ndim == 3 and img.shape[2] == 4:
        return cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    
    return img

<p style="font-size:16px">
   위 함수는 이미지를 입력 받았을 때, 버그를 최소화하는 장치이다. cv2 모듈로 이미지를 불러왔을 때, 이미지의 특성 채널 개수가 동일하지 않다면 'shape error'가 출력되면서 작동을 하지 않게되므로, 흑백 이미지와 같이 2채널인 경우에는 강제적으로 3채널 BGR로, 투명성이 존재하는 이미지처럼 알파(alpha) 채널이 존재하는 4채널인 경우에도 강제적으로 3채널 BGR로 변환한다.

</p><br>

In [43]:
def bgr_to_rgb(img_bgr: np.ndarray) -> np.ndarray:
    img_rgb = (np.flip(img_bgr, axis=-1)).astype(np.float32) / 255
    return img_rgb

def rgb_to_luma(rgb: np.ndarray) -> np.ndarray: # BT.601 근사 
    return (0.299*rgb[...,0] + 0.587*rgb[...,1] + 0.114*rgb[...,2]).astype(np.float32) 

<p style="font-size:16px">
   cv2 모듈의 imread() 함수는 이미지를 불러올 때, 항상 bgr 코드로 불러오는데, 특성을 왜곡없이 추출하고 평가 지표를 계산하기 위해 이를 rgb 코드로 변환을 해야한다.
   이러한 변환을 통해, <code>rgb_to_luma</code> 함수로 rgb 코드에 BT.601에 근거하는 계수를 곱하여 이미지의 밝기를 강조한다.
   
</p><br>

In [44]:
def make_lq_from_hq(hq_bgr: np.ndarray, scale: float, blur_sigma: float) -> np.ndarray:
    img_height, img_width = hq_bgr.shape[:2]

    to_width = max(1, int(img_width * scale))
    to_height = max(1, int(img_height * scale))

    down = cv2.resize(hq_bgr, (to_width, to_height), interpolation=cv2.INTER_AREA)
    up   = cv2.resize(down, (img_width, img_height), interpolation=cv2.INTER_LINEAR)

    if blur_sigma > 0:
        blur_img = cv2.GaussianBlur(up, (0,0), sigmaX=blur_sigma, sigmaY=0)
    else:
        blur_img = up
    return blur_img

<p style="font-size:16px">
   위 함수는 훈련에 사용할 이미지를 정제하는 과정이다. HQ(고화질) 이미지를 강제적으로 scale 비율만큼 Down-Scaling을 실행한 다음에 다시 원상복구를 하면 LQ(저화질) 이미지가 추출된다.

   축소할 때에는 interpolation 옵션에 cv2 모듈의 INTER_AREA 옵션을 주어 주변 픽셀 값들의 평균치로 축소했으며, aliasing 문제 또한 해결했고, 확대할 때에는 INTER_LINEAR 옵션을 주어 2x2 픽셀을 채우면서 블러 처리를 주었다.

   그러나 이러한 다운샘플링과 업샘플링 과정만으로는 자연스러운 이미지를 만들 수 없었고, 경계선이 여전히 뚜렷하게 남으며 노이즈도 약간 발생하는 것을 발견했다.

   따라서 고주파 성격을 띄는 경계선을 더욱 흐릿하게 블러처리하기 위해서 고주파를 억제시키는 가우시안 블러(Gaussian Blur)을 사용했다.

   cv2 모듈은 이 가우시안 블러 함수를 제공한다. 시그마(sigma) 값을 가중치로 두어 블러의 정도를 결정하는데, 컷오프(Cut-off) 주파수가 낮아지면서 고주파 영역을 더 많이 필터링을 하게 된다. 처음에는 그리드서치(Grid-Search)를 활용한 브루트포스(Brute-Force)로 최적의 시그마 값을 추출해내려고 했으나, 실험 결과 시그마 값은 0.8~1.2 사이에서 품질 변화가 미미했다.

   핵심 품질에 직접적인 영향을 주는 것이 아니기도 하며, 탐색 공간을 축소함으로써 연산량을 줄이기 위해 시그마 값을 1.0으로 고정했다.
</p>

<p style="font-size:20px">
   <mark>가우시안(Gaussian) 필터란?</mark>
</p>
<p style="font-size:16px">
   가우시안(Gaussian) 필터는 실제 통신에서도 고주파를 억제하면서 노이즈를 상쇄시키기 위해 많이 사용되는 필터이다. 저주파 영역에서는 큰 가중치가 존재하지만, 고주파 영역으로 이동할수록 가중치가 점점 감소하는 그래프 구조를 가지고 있다.
</p>

<p style="font-size:20px">
   <mark>Butterworth 필터와 Laplacian 필터를 사용하지 않은 이유</mark>
</p>
<p style="font-size:16px">
   Butterworth 필터는 cut-off 구간이 짧아 더 날카롭게 필터링이 되지만, 그러한 특성으로 인해 물결현상(Ringing Effect)이 존재하여 이미지가 밝고 어두운 패턴이 반복적으로 나타나게 된다. Laplacian 필터는 2차 미분을 하기 때문에 작은 밝기의 변화더라도 강하게 강조되어 펄스(Pulse) 노이즈가 발생한다. 따라서 이미지 보정에서는 물결현상과 노이즈를 방지하는 가우샨(Gasussian) 필터가 훨씬 더 자주 사용된다.
</p><br>

In [ ]:
def grid_search_for_image(lq_rgb: np.ndarray, hq_rgb: np.ndarray) -> np.ndarray:
    lq_rgb_float32 = lq_rgb.astype(np.float32)
    hq_rgb_float32 = hq_rgb.astype(np.float32)
    patch = params.patch
    stride = params.stride
    k_values = params.k_values
    blur = cv2.GaussianBlur(lq_rgb_float32, (0, 0), sigmaX=2.0, sigmaY=0)
    detail = lq_rgb_float32 - blur
    height, width = lq_rgb_float32.shape[:2]
    r = patch // 2
    kmap = np.zeros((height, width), dtype=np.float32)

    def psnr_float(a: np.ndarray, b: np.ndarray, eps=1e-12) -> float:
        mse = float(np.mean((a - b) ** 2))
        return 100.0 if mse <= eps else 10.0 * np.log10(1.0 / mse)

    for y in range(r, height - r, stride):
        for x in range(r, width - r, stride):
            ys, ye, xs, xe = y - r, y + r + 1, x - r, x + r + 1
            I_p, GT_p, det_p = lq_rgb_float32[ys:ye, xs:xe], hq_rgb_float32[ys:ye, xs:xe], detail[ys:ye, xs:xe]
            best_k, best_ps = 0.0, -1e9
            for k in k_values:
                Sh_p = np.clip(I_p + k * det_p, 0.0, 1.0)
                ps = psnr_float(Sh_p, GT_p)
                if ps > best_ps:
                    best_ps, best_k = ps, k
            kmap[y, x] = best_k
            
    kmap = cv2.GaussianBlur(kmap, (0, 0), sigmaX=float(stride))
    kmin, kmax = float(min(k_values)), float(max(k_values))
    kmap = np.clip(kmap, kmin, kmax)
    return kmap

<p style="font-size:16px">
   위 함수는 이미지를 정제하기 위한 하이퍼파라미터 값을 그리드 교차 검증을 통해 찾는 함수이다.

   하이퍼파라미터를 불러오고 가우샨 블러로 저주파 부분을 얻어낸 이미지를 blur에 저장하고, 원본 이미지에서 저주파 부분을 얻어낸 이미지를 빼서 고주파 부분을 얻어낸 이미지를 detail에 저장한다.

   이 후, 그리드서치를 본격적으로 진행한다. r은 패치의 반경을 의미하며, 이미지 경계의 4픽셀부터 반대편 이미지 경계의 4픽셀 전의 위치까지 stride만큼 이동하면서 k값 후보군을 넣어보면서 결과를 비교한다.

   각 패치의 PSNR 값을 비교하면서 가장 높은 PSNR 수치를 기록하는 k갑을 찾아내고 해당 값을 k맵에 저장하면, 이미지의 각 위치마다 가장 품질이 좋게 복원되는 샤프닝 강도를 찾아낼 수 있다.

   PSNR을 찾아낼 때는 원본 이미지의 패치에 샤프닝 강도로 고주파 이미지의 패치를 적용하여 고주파 이미지의 패치와 보정된 이미지의 패치를 비교해서 더 높은 PSNR 수치를 가진 것을 저장하는데, 이 때 패치의 크기를 0~1로 제한하였다. 샤프닝 강도가 1.0을 넘어가게 되면, MSE(Mean Squared Error)가 급격하게 증가하게 되면서 PSNR 수치도 0dB로 급격하게 낮아질 수 있다. 따라서 샤프닝 강도의 값과 상관없이 안정적으로 PSNR 수치를 찾아내도록 0, 1의 범위로 클리핑하였다.
</p>

<p style="font-size:20px">
   <mark>PSNR(Peak Signal-to-Noise Ratio)이란?
</p>
<p style="font-size:16px">
   영상 처리에서 사용되는 원본 영상과 복원된 영상의 유사도 및 품질을 나타내는 지표이다.

   ```PSNR = 10 [log(MAX_I^2) - log(MSE​)]```
   
   PSNR은 MSE 오차를 기반으로, 원본 신호와 노이즈(오차)의 비율을 로그나 데시벨로 표현한다.   
</p><br>

In [46]:
def extract_features(patch_luma: np.ndarray) -> Dict[str, float]:
    mean, std = float(np.mean(patch_luma)), float(np.std(patch_luma))
    f = np.fft.fft2(patch_luma)
    fshift = np.fft.fftshift(f)
    mag_spec = np.abs(fshift)

    rows, cols = patch_luma.shape
    crow, ccol = rows // 2, cols // 2

    p = 0.25
    lr, lc = int(rows * p), int(cols * p)

    total_energy = float(np.sum(mag_spec))
    high_freq_energy = total_energy - float(np.sum(mag_spec[crow-lr:crow+lr, ccol-lc:ccol+lc]))
    high_freq_ratio = high_freq_energy / total_energy if total_energy > 1e-6 else 0.0
    return {"mean": mean, "std": std, "high_freq_ratio": high_freq_ratio}

<p style="font-size:16px">
위 함수는 그리드서치를 한 샤프닝 강도를 어떤 특징을 가진 패치에 적용을 해야하는지를 판단하기 위한 함수이다.

입력 값으로는 저화질(LQ) 이미지의 RGB 코드를 밝기 값으로 변환하는 rgb_to_luma 함수의 반환값이 들어간다.
    
patch_luma 배열에는 한 패치의 밝기 값이 저장이 되어있고, 각 픽셀의 평균과 표준편차를 구하여 밝기와 명암 정도를 저장한다.

훈련에 들어갈 특징(Feature)은 평균과 표준편차 말고도 해당 패치가 평평한지, 복잡한지를 비율로 나타내는 고주파 비율이 들어가야 한다.

비율을 구하기 위해서 패치의 밝기 값을 2D 고속 푸리에 변환을 하여 주파수 스펙트럼을 바꿔준 후, fftshift() 함수로 저주파를 중앙으로, 고주파를 주변 영역으로 이동시켜주고 절댓값을 취해준다. 그러면 스펙트럼의 세기(Magnitude)를 얻어내어 해당 패치의 픽셀별 주파수 진폭을 알아낼 수 있다.

고주파 비율을 추출하기 위해서는 전체 에너지에서 저주파 에너지를 빼면 구할 수 있기 때문에 저주파 영역과 고주파 영역을 분리해야 한다. fftshift로 저주파를 중앙으로 옮겼기 때문에 crow(저주파 행 인덱스)와 ccol(저주파 열 인덱스)를 패치 중앙으로 할당하고, 25%로 스케일링한다.

25%로 스케일링을 하는 이유는 패치의 중앙으로부터 어느 픽셀까지를 저주파 영역으로 간주할지를 지정하기 위해서다. 비율이 커지면 저주파로 인식하는 픽셀이 많아져 고주파 비율이 낮아지고 경계가 모호해진다. 반대로 비율이 낮아지면 저주파로 인식하는 픽셀이 적어져 고주파 비율이 커지면서 평평한 부분까지 샤프닝 강도가 강해질 수 있으므로, 20%~30% 정도로 스케일링하는 것이 좋다.

마지막으로, 패치별 평균 밝기와, 밝기 분포 폭, 고주파 비율을 반환하여 훈련 세트에 넣을 특징(Features)들을 반환한다.
</p><br>

In [47]:
def generate_dataset() -> str:
    os.makedirs(params.output_dir, exist_ok=True)

    hq_paths = list_images(params.train_dir, params.img_extensions)
    if not hq_paths:
        print(f"[INFO] Can't find image: '{params.train_dir}'")
        return None
        
    all_patch_data = []
    print("[INFO] Creating Data Set...")
    for hq_path in tqdm(hq_paths, desc="Processing Images"):
        hq_bgr = ensure_3ch(cv2.imread(hq_path))
        lq_bgr = make_lq_from_hq(hq_bgr, params.DOWNSCALE, params.LQ_BLUR_SIGMA)
        hq_rgb, lq_rgb = bgr_to_rgb(hq_bgr), bgr_to_rgb(lq_bgr)
        k_map = grid_search_for_image(lq_rgb, hq_rgb)

        height, width = lq_rgb.shape[:2]
        stride, r = params.stride, params.patch // 2
        lq_luma = rgb_to_luma(lq_rgb)
        
        for y in range(r, height - r, stride):
            for x in range(r, width - r, stride):
                patched = lq_luma[y-r:y+r+1, x-r:x+r+1]
                features = extract_features(patched)
                features['target_k'] = k_map[y, x]
                all_patch_data.append(features)

    df = pd.DataFrame(all_patch_data)
    output_path = os.path.join(params.output_dir, "training_dataset.csv")
    df.to_csv(output_path, index=False)
    print(f"[INFO] Successfully created. : {output_path}")
    return output_path

<p style="font-size:16px">
    위 함수는 데이터셋 파일을 만들어내는 데 사용한다. 위에서 만들었던 함수들을 활용하여, 그리드서치 후, 패치별 특징(features) 값들을 추출해내어 저장하고 target_k라는 타겟 열을 만들어 해당 피쳐값들에 해당하는 샤프닝강도 k맵에서 가져와 저장한다.
</p>

<p style="font-size:16px">
</p><br>

<br><hr>

<h2> 5. 결론 </h2>
<p style="font-size:16px">
    - 그리드서치를 PSNR 지표를 활용하여 진행하면서 최적의 샤프닝 강도 k맵을 생성<br>
    - 이미지의 밝기를 주파수 도메인에서 해석 및 변형하여 각 패치의 특징(밝기, 변화 폭, 고주파 비율)을 추출<br>
    - 이미지를 다운스케일링과 업스케일링을 통해 해상도를 낮추고 가우샨(Gaussian) 필터를 활용하여 저주파 영역만 추출된 블러 이미지를 생성 및 훈련 세트 전처리<br><br>

이미지의 패치 별 최적의 샤프닝 강도 배열과 밝기, 밝기 변화 진폭, 고주파수 비율을 훈련 세트로 한 데이터셋을 생성하였으므로, 본격적으로 XGBoost 모델로 지도 학습을 진행하는 것을 진행할 예정이다.

    -> notebooks/model.ipynb
</p>